# Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

# Overview of the data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Exploring genres

In [3]:
%%sql
WITH nation as(
 Select *
 from invoice i
 where billing_country=='USA'),
link as
( 
    select *
    from invoice_line il
    inner join nation n on n.invoice_id=il.invoice_id
    inner join track t on t.track_id=il.track_id
    )
select g.name,count(l.track_id) no_of_tracks_sold,
      Cast(count(l.track_id) AS FLOAT)/ ( select count(*) from link) perc
    from genre g
    inner join link l on l.genre_id=g.genre_id
    group by 1
    order by 2 desc
    LIMIT 10;

 * sqlite:///chinook.db
Done.


name,no_of_tracks_sold,perc
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Rock genre outshines the other genre with 53% of sales. While other genres has sales less than 13% which is much lower compared to rock genre.

# Exploring artists

In [4]:
%%sql
select a.name
from genre g
inner join track t on t.genre_id=g.genre_id
inner join album al on al.album_id=t.track_id
inner join artist a on a.artist_id=al.artist_id
where g.name=='Rock'
Limit 5;

 * sqlite:///chinook.db
Done.


name
AC/DC
Accept
Accept
AC/DC
Aerosmith


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:
 * AC/DC
 * Accept
 * Aerosmith

# Analyzing Employee Sales Performance

In [5]:
%%sql
select e.first_name ||''|| e.last_name name,
        sum(i.total) total
from employee e
inner join customer c on c.support_rep_id=e.employee_id
inner join invoice i on i.customer_id=c.customer_id
group by 1

 * sqlite:///chinook.db
Done.


name,total
JanePeacock,1731.5100000000039
MargaretPark,1584.0000000000034
SteveJohnson,1393.920000000002


There is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee).

# Analyzing Sales by Country

In [6]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )
SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


USA has highest number of sales which is nearly double of second place Canada. 

# Analyzing Albums versus Individual Tracks Performance

In [7]:
%%sql
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;


 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.